In [ ]:
import pandas as pd
dados_preProcessado = pd.read_csv('https://raw.githubusercontent.com/diego2017003/knn_treeClassifier/main/dados_preProcessados.csv')
dados_originais = pd.read_csv('https://raw.githubusercontent.com/diego2017003/knn_treeClassifier/main/heart.csv')
dados_preProcessado.drop(columns=['Unnamed: 0'],inplace=True)
dados_originais.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
dados_preProcessado.head()

,age,sex,trtbps,chol,fbs,thalachh,exng,oldpeak,slp_0,slp_1,slp_2,thall_0,thall_1,thall_2,thall_3,caa_0,caa_1,caa_2,caa_3,caa_4,restecg_Normal,restecg_Possui_anormalidade,restecg_Provável_hipertrofia,cp_Angina_atipica,cp_Angina_tipica,cp_Assintomatico,cp_Não_Angina,restecg_Possui_anormalidades,output
0,0.952197,1,0.763956,-0.256334,1,0.015443,0,1.087338,1,0,0,0,1,0,0,1,0,0,0,0,1,-1.003306,0,0,0,0,1,0,1
1,-1.915313,1,-0.092738,0.072199,0,1.633471,0,2.122573,1,0,0,0,0,1,0,1,0,0,0,0,0,0.996705,0,1,0,0,0,1,1
2,-1.474158,0,-0.092738,-0.816773,0,0.977514,0,0.310912,0,0,1,0,0,1,0,1,0,0,0,0,1,-1.003306,0,0,1,0,0,0,1
3,0.180175,1,-0.663867,-0.198357,0,1.239897,0,-0.206705,0,0,1,0,0,1,0,1,0,0,0,0,0,0.996705,0,0,1,0,0,1,1
4,0.290464,0,-0.663867,2.082050,0,0.583939,1,-0.379244,0,0,1,0,0,1,0,1,0,0,0,0,0,0.996705,0,0,0,1,0,1,1


Este notebook terá foco na utilização de comitês para a base de dado que vem sendo trabalhada durante o projeto.     

##1. **Boost**
---

### 1.1 MLPClassifier 

#### Dados originais

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
import numpy as np

class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

 
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
mlpBoost_10 = AdaBoostClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7,algorithm='SAMME')
mlpBoost_15 = AdaBoostClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7,algorithm='SAMME')
mlpBoost_20 = AdaBoostClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7,algorithm='SAMME')


In [ ]:
'''conda update anaconda-navigator
anaconda-navigator --reset
conda update anaconda-client
conda update -f anaconda-client'''

'conda update anaconda-navigator\nanaconda-navigator --reset\nconda update anaconda-client\nconda update -f anaconda-client'

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
import math
#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mlpBoost_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=mlpBoost_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=mlpBoost_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8302242296746941 std: 0.08148934320862557
n = 15 Precisão média :0.8162749466735535 std: 0.08130395367961846
n = 20 Precisão média :0.8042343604108311 std: 0.05669974628070068


#### Dados Processados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mlpBoost_p_10 = AdaBoostClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=10, random_state=7,algorithm='SAMME')
mlpBoost_p_15 = AdaBoostClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=15, random_state=7,algorithm='SAMME')
mlpBoost_p_20 = AdaBoostClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=20, random_state=7,algorithm='SAMME')
#mlpBoost_p_10 = AdaBoostClassifier(base_estimator = customMLPClassifer(activation='identity', hidden_layer_sizes=(12,), solver='adam'),n_estimators=10, random_state=7,algorithm='SAMME')
#mlpBoost_p_15 = AdaBoostClassifier(base_estimator = customMLPClassifer(activation='identity', hidden_layer_sizes=(12,), solver='adam'),n_estimators=15, random_state=7,algorithm='SAMME')
#mlpBoost_p_20 = AdaBoostClassifier(base_estimator = customMLPClassifer(activation='identity', hidden_layer_sizes=(12,), solver='adam'),n_estimators=20, random_state=7,algorithm='SAMME')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
'''param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,),(22,),(23,),(24,),
             (25,),(26,),(27,),(28,),(29,),(30,),(31,),(32,),(33,),(34,),(35,)
             ]
        }
       ]
clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,scoring='accuracy')

clf.fit(X,y)


print("Best parameters set found on development set:")
print(clf.best_params_)'''

'param_grid = [\n        {\n            \'activation\' : [\'identity\', \'logistic\', \'tanh\', \'relu\'],\n            \'solver\' : [\'lbfgs\', \'sgd\', \'adam\'],\n            \'hidden_layer_sizes\': [\n             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,),(22,),(23,),(24,),\n             (25,),(26,),(27,),(28,),(29,),(30,),(31,),(32,),(33,),(34,),(35,)\n             ]\n        }\n       ]\nclf = GridSearchCV(MLPClassifier(), param_grid, cv=3,scoring=\'accuracy\')\n\nclf.fit(X,y)\n\n\nprint("Best parameters set found on development set:")\nprint(clf.best_params_)'

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=mlpBoost_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=mlpBoost_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=mlpBoost_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))
#n = 10 Precisão média :0.852618703965453 std: 0.05008061533940598
#n = 15 Precisão média :0.8335730682286411 std: 0.06789153517380564
#n = 20 Precisão média :0.8373666868906808 std: 0.06344416567018743

n = 10 Precisão média :0.8384491514677272 std: 0.060535771479765105
n = 15 Precisão média :0.8630253851325833 std: 0.06231037294910893
n = 20 Precisão média :0.8608061114582013 std: 0.05086677557434708


### 1.2 Naive Bayes
---

#### Dados originais

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
nbBoost_10 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7,algorithm='SAMME')
nbBoost_15 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7,algorithm='SAMME')
nbBoost_20 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7,algorithm='SAMME')

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
import math
#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=nbBoost_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=nbBoost_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=nbBoost_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8293866872291795 std: 0.07032541790351075
n = 15 Precisão média :0.839855330428086 std: 0.07196711973407324
n = 20 Precisão média :0.839855330428086 std: 0.07196711973407324


#### Dados Processados 

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
nbBoost_p_10 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7,algorithm='SAMME')
nbBoost_p_15 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7,algorithm='SAMME')
nbBoost_p_20 = AdaBoostClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7,algorithm='SAMME')

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=nbBoost_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=nbBoost_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=nbBoost_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8040560408859474 std: 0.09210672977281531
n = 15 Precisão média :0.8040560408859474 std: 0.09210672977281531
n = 20 Precisão média :0.8040560408859474 std: 0.09210672977281531


## 2. Bagging
---

### 2.1 MLP 

#### Dados originais

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
mlpBag_10 = BaggingClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7)
mlpBag_15 = BaggingClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7)
mlpBag_20 = BaggingClassifier(base_estimator = customMLPClassifer(),n_estimators=10, random_state=7)


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
import math
#scoring = ['precision_macro', 'recall_macro']
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mlpBag_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=mlpBag_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=mlpBag_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8377009052608898 std: 0.06960447111959236
n = 15 Precisão média :0.8420482744379804 std: 0.06135139587538859
n = 20 Precisão média :0.8326179348257522 std: 0.07905223835649483


#### Dados PreProcessados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mlpBag_p_10 = BaggingClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=10, random_state=7)
mlpBag_p_15 = BaggingClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=15, random_state=7)
mlpBag_p_20 = BaggingClassifier(base_estimator = customMLPClassifer(alpha=0.0001,learning_rate='constant',activation='identity', hidden_layer_sizes=(21,), solver='sgd'),n_estimators=20, random_state=7)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=mlpBag_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=mlpBag_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=mlpBag_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8444788680102148 std: 0.05481679205089955
n = 15 Precisão média :0.8437595782700271 std: 0.049509367139697884
n = 20 Precisão média :0.843501372423586 std: 0.05807702840628681


### 2.2 Naive Bayes

#### Dados originais

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
nbBag_10 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7)
nbBag_15 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7)
nbBag_20 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=nbBag_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=nbBag_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=nbBag_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8314622484069079 std: 0.059105151157382955
n = 15 Precisão média :0.8184218206973626 std: 0.05917025228746715
n = 20 Precisão média :0.8092972099280148 std: 0.05056133001621758


#### Dados PreProcessados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
nbBag_p_10 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=10, random_state=7)
nbBag_p_15 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=15, random_state=7)
nbBag_p_20 = BaggingClassifier(base_estimator = GaussianNB(),n_estimators=20, random_state=7)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=nbBag_p_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=nbBag_p_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=nbBag_p_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8034839180427416 std: 0.07957654234288679
n = 15 Precisão média :0.8098564793050087 std: 0.07728274294416708
n = 20 Precisão média :0.8169193878017408 std: 0.0742722387493331


### 3. Stacking Homogêneo
---

#### MLP

#### Dados originais

In [ ]:
from sklearn.model_selection import train_test_split
base_learners_10 = []
base_learners_15 = []
base_learners_20 = []
for i in range(10):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('rf_'+str(i), MLPClassifier()))

for i in range(15):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_15.append(('mlp_'+str(i), MLPClassifier()))

for i in range(20):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_20.append(('mlp_'+str(i), MLPClassifier()))

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np


X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

mlpStk_10 = StackingClassifier(estimators = base_learners_10,cv=10)
mlpStk_15 = StackingClassifier(estimators = base_learners_15,cv=10)
mlpStk_20 = StackingClassifier(estimators = base_learners_20,cv=10)


In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mlpStk_10, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_15 = cross_validate(estimator=mlpStk_15, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_20 = cross_validate(estimator=mlpStk_20, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8145623038636469 std: 0.05662330796770682
n = 15 Precisão média :0.806052406450274 std: 0.046400642083073304
n = 20 Precisão média :0.8099110302608755 std: 0.04683408043536126


#### Dados PreProcessados

In [ ]:
X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mlpStk_p_10 = StackingClassifier(estimators = base_learners_10,cv=10)
mlpStk_p_15 = StackingClassifier(estimators = base_learners_15,cv=10)
mlpStk_p_20 = StackingClassifier(estimators = base_learners_20,cv=10)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=mlpStk_p_10, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_p_15 = cross_validate(estimator=mlpStk_p_15, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_p_20 = cross_validate(estimator=mlpStk_p_20, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8102504978645134 std: 0.04952702398672045
n = 15 Precisão média :0.7995353189838483 std: 0.03327987757151561
n = 20 Precisão média :0.8055751391466434 std: 0.04219952646098446


### 3.2 Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
base_learners_10 = []
base_learners_15 = []
base_learners_20 = []
for i in range(10):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('nb_'+str(i), GaussianNB()))

for i in range(15):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_15.append(('nb_'+str(i), GaussianNB()))

for i in range(20):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_20.append(('nb_'+str(i), GaussianNB()))

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

nbStk_10 = StackingClassifier(estimators = base_learners_10,cv=10)
nbStk_15 = StackingClassifier(estimators = base_learners_15,cv=10)
nbStk_20 = StackingClassifier(estimators = base_learners_20,cv=10)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=nbStk_10, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_15 = cross_validate(estimator=nbStk_15, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)
scores_20 = cross_validate(estimator=nbStk_20, X= X, y= y, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_15 = np.array(scores_15['test_prec_macro']).mean()
std_15 = np.array(scores_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_15)+ " std: "+str(std_15))
mean_20 = np.array(scores_20['test_prec_macro']).mean()
std_20 = np.array(scores_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_20)+ " std: "+str(std_20))

n = 10 Precisão média :0.8092750470633596 std: 0.06355865857774357
n = 15 Precisão média :0.8092750470633596 std: 0.06355865857774357
n = 20 Precisão média :0.8092750470633596 std: 0.06355865857774357


#### Dados Preprocessados

In [ ]:
nbStk_p_10 = StackingClassifier(estimators = base_learners_10,cv=10)
nbStk_p_15 = StackingClassifier(estimators = base_learners_15,cv=10)
nbStk_p_20 = StackingClassifier(estimators = base_learners_20,cv=10)

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_p_10 = cross_validate(estimator=nbStk_10, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_15 = cross_validate(estimator=nbStk_15, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)
scores_p_20 = cross_validate(estimator=nbStk_20, X= X_p, y= y_p, scoring=scoring,cv=10, return_train_score=True)

In [ ]:
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))
mean_p_15 = np.array(scores_p_15['test_prec_macro']).mean()
std_p_15 = np.array(scores_p_15['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_15)+ " std: "+str(std_p_15))
mean_p_20 = np.array(scores_p_20['test_prec_macro']).mean()
std_p_20 = np.array(scores_p_20['test_prec_macro']).std()
print("n = 20 Precisão média :"+str(mean_p_20)+ " std: "+str(std_p_20))

n = 10 Precisão média :0.8032499876714274 std: 0.07280909439688978
n = 15 Precisão média :0.8032499876714274 std: 0.07280909439688978
n = 20 Precisão média :0.8032499876714274 std: 0.07280909439688978


### 4. Stacking heterogêneo  

ad+mlp

In [ ]:
from sklearn.tree import DecisionTreeClassifier

base_learners_10 = []

for i in range(5):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i), DecisionTreeClassifier()))
for i in range(5):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('mlp_'+str(i), MLPClassifier()))

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

mdStk_10 = StackingClassifier(estimators = base_learners_10,cv=10)


X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mdStk_p_10 = StackingClassifier(estimators = base_learners_10,cv=10)


In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mdStk_10, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_p_10 = cross_validate(estimator=mdStk_p_10, X= X_p, y= y_p, scoring=scoring,cv=5, return_train_score=True)


In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))


n = 10 Precisão média :0.8522364252104964 std: 0.07203744448176708
n = 15 Precisão média :0.8363776299380016 std: 0.05180095063579365


mlp + nb

In [ ]:
base_learners_10 = []

for i in range(5):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i), GaussianNB()))
for i in range(5):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('mlp_'+str(i), MLPClassifier()))

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

mnStk_10 = StackingClassifier(estimators = base_learners_10,cv=10)


X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
mnStk_p_10 = StackingClassifier(estimators = base_learners_10,cv=10)


In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=mnStk_10, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_p_10 = cross_validate(estimator=mnStk_p_10, X= X_p, y= y_p, scoring=scoring,cv=5, return_train_score=True)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))

nb + dt

In [ ]:
base_learners_10 = []

for i in range(5):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('dt_'+str(i), GaussianNB()))
for i in range(5):
  #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
  base_learners_10.append(('mlp_'+str(i), DecisionTreeClassifier()))

In [ ]:
X = dados_originais.drop(columns=['output'])
y = dados_originais.output
base_learners = [('rf_1', MLPClassifier())]

dnStk_10 = StackingClassifier(estimators = base_learners_10,cv=10)


X_p = dados_preProcessado.drop(columns=['output'])
y_p = dados_preProcessado.output
dnStk_p_10 = StackingClassifier(estimators = base_learners_10,cv=10)


In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=dnStk_10, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_p_10 = cross_validate(estimator=dnStk_p_10, X= X_p, y= y_p, scoring=scoring,cv=5, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))

NB+MLP+DT

In [ ]:
scoring = {'prec_macro': 'precision_macro',
            'rec_macro': make_scorer(recall_score, average='macro')}
scores_10 = cross_validate(estimator=dnStk_10, X= X, y= y, scoring=scoring,cv=5, return_train_score=True)
scores_p_10 = cross_validate(estimator=dnStk_p_10, X= X_p, y= y_p, scoring=scoring,cv=5, return_train_score=True)

In [ ]:
mean_10 = np.array(scores_10['test_prec_macro']).mean()
std_10 = np.array(scores_10['test_prec_macro']).std()
print("n = 10 Precisão média :"+str(mean_10)+ " std: "+str(std_10))
mean_p_10 = np.array(scores_p_10['test_prec_macro']).mean()
std_p_10 = np.array(scores_p_10['test_prec_macro']).std()
print("n = 15 Precisão média :"+str(mean_p_10)+ " std: "+str(std_p_10))